# CIDAUT AI Fake Scene Classification 2024

---
author: Brice Piaple Dada
date: December 21, 2024
---

## Unzip the data

In [ ]:
#| echo: true


import zipfile

# Path to the .zip file
zip_file_path = 'Data/cidaut-ai-fake-scene-classification-2024.zip'

# Path toi extract teh contents
extract_path = 'Data/Unzip_data'

# Open the .zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip:
    zip.extractall(extract_path)
    
    print("Extractiob Complete.")


## Import our libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

## Load our data

###  Load our csv and images files

In [4]:
def load_image(image_path) -> tf.Tensor:
    """Loads an image from the specified file path, decodes it as a JPEG, resizes it to 224x224 pixels,
    and normalizes the pixel values to the range [0, 1].
    
    Args:
        image_path (str): The file path to the image.

    Returns:
        tf.Tensor: A tensor representing the processed image.
    """
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image
